In [89]:
import torch.nn as nn
import torch
import torch.optim as optim

In [80]:
t_c = torch.tensor([0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])
t_c = t_c.unsqueeze(1)
t_u = t_u.unsqueeze(1)
t_u.shape

torch.Size([11, 1])

In [81]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 3,  5, 10,  0,  8,  4,  1,  9,  6]), tensor([2, 7]))

In [82]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

t_un_train, t_un_val

(tensor([[8.1900],
         [4.8900],
         [6.8400],
         [3.5700],
         [4.8400],
         [5.6300],
         [5.5900],
         [6.0400],
         [3.3900]]),
 tensor([[5.8200],
         [2.1800]]))

In [85]:
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[0.6987],
        [0.8597]], grad_fn=<AddmmBackward0>)

In [90]:
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2)

In [92]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [93]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [95]:
training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 2.9522, Validation loss 3.8228
Epoch 1000, Training loss 2.9386, Validation loss 4.3708
Epoch 2000, Training loss 2.9375, Validation loss 4.5367
Epoch 3000, Training loss 2.9374, Validation loss 4.5850

Parameter containing:
tensor([[5.6295]], requires_grad=True)
Parameter containing:
tensor([-19.0253], requires_grad=True)
